## 搭建 ResNet34 网络

In [1]:
from torch import nn
import torch as t
from torch.nn import functional as F
from torch.autograd import Variable as V

In [2]:
class ResidualBlock(nn.Module):
    """
    实现子module: Residual Block
    """
    def __init__(self, inchannel, outchannel, stride=1, shortcut=None):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(in_channels=inchannel, out_channels=outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=outchannel, out_channels=outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.right = shortcut
    
    def forward(self, x):
        out = self.left(x)
        residual = x if self.right is None else self.right(x)
        out += residual
        return F.relu(out)

In [3]:
class ResNet(nn.Module):
    """
    实现主module: ResNet34
    ResNet34 包含多个layer, 每个layer又包含多个residual block
    用子module来实现residual block, 用 _make_layer 函数来实现layer
    """
    def __init__(self, num_classes=1000):
        super(ResNet, self).__init__()
        # 前几层图像转换
        self.pre = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        # 重复的layer, 分别有3, 4, 6, 3个residual block
        self.layer1 = self._make_layer(64, 64, 3)
        self.layer2 = self._make_layer(64, 128, 4, stride=2)
        self.layer3 = self._make_layer(128, 256, 6, stride=2)
        self.layer4 = self._make_layer(256, 512, 3, stride=2)
        
        # 分类用的全连接
        self.fc = nn.Linear(512, num_classes)
    
    def _make_layer(self, inchannel, outchannel, block_num, stride=1):
        """
        构建layer, 包含多个residual block
        """
        shortcut = nn.Sequential(
            nn.Conv2d(in_channels=inchannel, out_channels=outchannel, kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        layers = []
        layers.append(ResidualBlock(inchannel, outchannel, stride, shortcut))
        
        for i in range(1, block_num):
            layers.append(ResidualBlock(outchannel, outchannel))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.pre(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = F.avg_pool2d(x, 7)
        x = x.view(x.size(0), -1)
        return self.fc(x)

In [4]:
model = ResNet()
input = V(t.randn(1, 3, 224, 224))
out = model(input)
out

tensor([[ 0.0929, -0.3950, -0.4729,  0.2882,  0.6009,  0.3002, -0.4543,  0.1153,
          0.0312,  0.4376,  0.1768, -0.2627, -0.2493, -0.1421,  0.2670,  0.2711,
         -0.0501, -0.3412,  0.1692, -0.4811,  0.6108,  0.3615, -0.1746, -0.2344,
         -0.0274, -0.2285, -0.1108,  0.1980,  0.2553, -0.1237, -0.3556, -0.2574,
          0.6607,  0.2911, -0.4118, -0.1778,  0.4043,  0.0296, -0.0504,  0.2192,
          0.5743,  0.4202,  0.3013,  0.1235,  0.2005,  0.0277, -0.6493,  0.0194,
         -0.5937, -0.1689, -0.1596, -0.2347, -0.0337, -0.0198, -0.5159, -0.1307,
          0.0596, -0.0886,  0.3157, -0.0831,  0.1781, -0.5370,  0.2013,  0.2319,
          0.1822,  0.2965,  0.4196,  0.0318, -0.0034, -0.1987,  0.3365,  0.3040,
         -0.0060,  0.1097, -0.4706,  0.2856, -0.4423,  0.3514,  0.0030,  0.2723,
         -0.0355,  0.4009,  0.0014,  0.3805,  0.5891, -0.4170,  0.7524, -0.3346,
          0.1126, -0.1750,  0.2317,  0.2606, -0.2347, -0.6723, -0.2562,  0.5006,
          0.4379,  0.1141,  